# Build a Preliminary BAL Sample for Shaw

In [1]:
# imports
import numpy as np

from astropy.coordinates import SkyCoord

from specdb.specdb import IgmSpec

from dla_cnn.io import load_ml_dr12

read_sets: Using set file -- 
  /home/xavier/local/Python/linetools/linetools/lists/sets/llist_v1.2.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/home/xavier/local/Python/linetools/linetools/lists/linelist.py:402: RuntimeWarning: divide by zero encountered in log10
  self._extra_table['log(w*f)'] = np.log10(qm_strength)


## Load igmsp

In [2]:
igmsp = IgmSpec()

Loading igmspec from /u/xavier/SPECDB/IGMspec_DB_v02.1.hdf5
Database is igmspec
Created on 2017-Sep-18
Version: v02.1


## Load DR12

In [3]:
dr12_slines, _ = load_ml_dr12()

In [4]:
dr12_slines[0:5]

Plate,Fiber,MJD,RA,DEC,zem,flg_BAL
int64,int64,int64,float64,float64,float64,int64
3586,16,55181,9.3398,-0.3678,2.23747383356,0
3586,38,55181,9.2152,-0.1659,2.19045149923,1
3586,90,55181,8.8235,-1.0162,2.15826554021,0
3586,114,55181,8.82,-0.33,2.22185597506,0
3586,158,55181,8.4806,-0.014,2.66399792728,1


### coords

In [5]:
dr12_coord = SkyCoord(ra=dr12_slines['RA'], dec=dr12_slines['DEC'], unit='deg')

## $z_{\rm em}$ cut

In [6]:
zcut = np.abs(dr12_slines['zem'] - 2.5) < 0.5

In [7]:
np.sum(zcut)

165087

## Without BAL

In [8]:
nobal = dr12_slines['flg_BAL'] == 0

In [9]:
dr12_nobal = dr12_slines[zcut & nobal][0:10000]
dr12_nobal[0:5]

Plate,Fiber,MJD,RA,DEC,zem,flg_BAL
int64,int64,int64,float64,float64,float64,int64
3586,16,55181,9.3398,-0.3678,2.23747383356,0
3586,90,55181,8.8235,-1.0162,2.15826554021,0
3586,114,55181,8.82,-0.33,2.22185597506,0
3586,188,55181,8.3748,-0.2188,2.21141635355,0
3586,228,55181,8.132,-0.4612,2.14094925525,0


In [10]:
idx_nobal = np.where(zcut & nobal)[0][0:10000]
len(idx_nobal)

10000

### Meta

In [11]:
dr12_coord[idx_nobal]

<SkyCoord (ICRS): (ra, dec) in deg
    [(   9.3398,  -0.3678), (   8.8235,  -1.0162), (   8.82  ,  -0.33  ),
     ..., ( 235.0636,  17.0328), ( 235.0029,  17.0446),
     ( 235.023 ,  17.3596)]>

In [12]:
matches_nobal, meta_nobal = igmsp.meta_from_coords(dr12_coord[idx_nobal], groups=['BOSS_DR12'])

Your search yielded 10000 matches from 10000 input coordinates
Final query yielded 10000 matches.


In [13]:
assert np.sum(matches_nobal) == 10000

### Spectra

In [14]:
spec_nobal = igmsp.spectra_from_meta(meta_nobal)

Staged 10000 spectra totalling 0.93 Gb
Loaded spectra


/home/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:249: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [36]:
for key in ['zem_GROUP','PLATE','FIBERID']:
    spec_nobal.meta[key] = np.array(meta_nobal[key])

In [37]:
spec_nobal.meta['PLATE']

array([3586, 3586, 3586, ..., 3932, 3932, 3932], dtype=int32)

#### Write

In [38]:
spec_nobal.write_to_binary_fits_table('DR12_noBAL.fits')

/home/xavier/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/numpy/ma/core.py:3008: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a structured
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  if (obj.__array_interface__["data"][0]
/home/xavier/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/numpy/ma/core.py:3009: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a structured
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  != se

Wrote spectrum to DR12_noBAL.fits


## With BAL

In [39]:
bal = dr12_slines['flg_BAL'] == 1

In [40]:
idx_bal = np.where(zcut & bal)[0][0:10000]
len(idx_bal)

10000

### Meta

In [41]:
matches_bal, meta_bal = igmsp.meta_from_coords(dr12_coord[idx_bal], groups=['BOSS_DR12'])

Your search yielded 10000 matches from 10000 input coordinates
Final query yielded 10000 matches.


### Spec

In [42]:
spec_bal = igmsp.spectra_from_meta(meta_bal)

Staged 10000 spectra totalling 0.93 Gb
Loaded spectra


/home/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:249: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [43]:
for key in ['zem_GROUP','PLATE','FIBERID']:
    spec_bal.meta[key] = np.array(meta_bal[key])

In [44]:
spec_bal.write_to_binary_fits_table('DR12_BAL.fits')

/home/xavier/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/numpy/ma/core.py:3008: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a structured
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  if (obj.__array_interface__["data"][0]
/home/xavier/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/numpy/ma/core.py:3009: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a structured
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  != se

Wrote spectrum to DR12_BAL.fits
